In [2]:
import pandas as pd
import tomllib

In [3]:
CONFIG_PATH = './data/config.toml'
def load_config(section_name: str) -> dict:
    with open(CONFIG_PATH, 'rb') as config_file:
        return tomllib.load(config_file)[section_name]

In [6]:
df = pd.read_csv('./data/raw_data/pop_census/base-cc-serie-historique-2020.CSV', sep=';', dtype='object')[list(load_config('population').keys())]
a = df.isna().any()
print(*a[a].index, sep='\n')

D82_LOG
D82_LOGVAC
DECE9099
DECE8290
NAIS9099
NAIS8290
D82_RP
D82_RSECOCC
